# with 语句和上下文管理器

## 简述

```python
# create/aquire some resource
...
try:
    # do something with the resource
    ...
finally:
    # destroy/release the resource
    ...
```
我们经常需要使用上面这样的代码形式，以确保资源的正常使用和释放。

`Python` 提供了 `with` 语句帮我们自动进行这样的处理

In [1]:
with open('my_file','w') as fp:
    data = fp.write("hello world")

In [2]:
# 等效于
fp = open("my_file",'w')
try:
    data = fp.write("hello world")
finally:
    fp.close()

## 上下文管理器

其基本用法如下：
```python
with <expression>:
    <block>
```
`<expression>` 执行的结果应当返回一个实现了上下文管理器的对象，即实现这样两个方法，`__enter__` 和 `__exit__`：

In [4]:
print fp.__enter__
print fp.__exit__

<built-in method __enter__ of file object at 0x04B17700>
<built-in method __exit__ of file object at 0x04B17700>


执行顺序`__enter__----><block>---->__exit__`

In [7]:
# 验证一下
class ContextManager(object):
    def __enter__(self):
        print "__enter__"
    def __exit__(self,exc_type, exc_value, traceback):
        print "__exit__"

In [8]:
with ContextManager():
    print "block"

__enter__
block
__exit__


In [9]:
# 假设block执行出错
with ContextManager():
    print 1/0

__enter__
__exit__


ZeroDivisionError: integer division or modulo by zero

## `__enter__` 的返回值

In [10]:
class ContextManager(object):
    
    def __enter__(self):
        print "Entering"
        # 这里有返回值
        return "my value"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Exiting"

In [11]:
# 返回值传给了value
with ContextManager() as value:
    print value

Entering
my value
Exiting


In [12]:
fp = open('my_file','r')
print fp.__enter__()
fp.close()

<open file 'my_file', mode 'r' at 0x04BFAA70>


In [13]:
import os 
os.remove('my_file')

In [17]:
# 将 __enter__ 的返回值设为这个上下文管理器对象本身
class ContextManager(object):
    def __enter__(self):
        print "enter"
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "exit"
with ContextManager() as value:
    print value

enter
exit


## 错误处理

上下文管理器对象将错误处理交给 `__exit__` 进行，可以将错误类型，错误值和 `traceback` 等内容作为参数传递给` __exit__` 函数：

In [27]:
class ContextManager(object):
    def __enter__(self):
        print 'enter'
        
    # exc_type是错误类型，exc.value是对应的错误信息
    def __exit__(self,exc_type,exc_value,traceback):
        print 'exit'
        if exc_type is not None:
            print 'Exception:',exc_value

In [29]:
with ContextManager():
    print 1/0

enter
exit
Exception: integer division or modulo by zero


ZeroDivisionError: integer division or modulo by zero

In [34]:
# 不让错误抛出：__exit__ 的返回值设为 True
class ContextManager(object):
    def __enter__(self):
        print 'enter'
        
    # exc_type是错误类型，exc_value是对应的错误信息
    def __exit__(self,exc_type,exc_value,traceback):
        print 'exit'
        if exc_type is not None:
            print 'Exception:',exc_type,exc_value
            # 把这个__exit__设置为True就不会抛出异常
            return True

In [35]:
with ContextManager():
    print 1/0

enter
exit
Exception: <type 'exceptions.ZeroDivisionError'> integer division or modulo by zero


## 数据库的例子

待深入学习：[数据库的例子](http://nbviewer.jupyter.org/github/lijin-THU/notes-python/blob/master/05-advanced-python/05.11-context-managers-and-the-with-statement.ipynb#%E6%95%B0%E6%8D%AE%E5%BA%93%E7%9A%84%E4%BE%8B%E5%AD%90)

## contextlib 模块

防止写入很多重复的模式，可以使用 `contextlib` 模块来进行处理。最简单的处理方式是使用 `closing` 函数确保对象的 `close()` 方法始终被调用

In [38]:
from contextlib import closing
import urllib

with closing(urllib.urlopen('http://www.baidu.com')) as url:
    html = url.read()

print html[:50]

<!DOCTYPE html>
<!--STATUS OK-->











In [39]:
# 另一个有用的方法是使用修饰符 @contextlib：
from contextlib import contextmanager

@contextmanager
def my_contextmanager():
    print "Enter"
    yield
    print "Exit"

with my_contextmanager():
    print "  Inside the with statement"

Enter
  Inside the with statement
Exit


`yield `之前的部分可以看成是 `__enter__` 的部分，yield 的值可以看成是 `__enter__` 返回的值，`yield `之后的部分可以看成是 `__exit__` 的部分。

In [40]:
@contextmanager
def my_contextmanager():
    print "Enter"
    yield "my value"
    print "Exit"
    
with my_contextmanager() as value:
    print value

Enter
my value
Exit


In [41]:
# 错误处理可以用 try 块来完成：
@contextmanager
def my_contextmanager():
    print "Enter"
    try:
        yield
    except Exception as exc:
        print "   Error:", exc
    finally:
        print "Exit"

In [42]:
with my_contextmanager():
    print 1/0

Enter
   Error: integer division or modulo by zero
Exit
